In [1]:
import warnings
import six
import numpy as np
import pandas as pd
from ppca import PPCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, FeatureAgglomeration, SpectralClustering, SpectralCoclustering, SpectralBiclustering, DBSCAN, AffinityPropagation, MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA, FastICA, IncrementalPCA, KernelPCA, FactorAnalysis, TruncatedSVD, SparsePCA, MiniBatchSparsePCA, DictionaryLearning, MiniBatchDictionaryLearning
from sklearn.manifold import TSNE, MDS, SpectralEmbedding, LocallyLinearEmbedding, Isomap
from umap import UMAP
import os
import tensorflow as tf
import tensorflow_hub as hub
from configparser import ConfigParser
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from align import hyper, srm, procrustes

global HYPERTOOLS_NOTEBOOK_MODE; HYPERTOOLS_NOTEBOOK_MODE = False

In [2]:
#debugging stuff
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
defaults = ConfigParser()
defaults.read('defaults.ini');

In [4]:
def hypertools_notebook_mode(toggle=None):
    global HYPERTOOLS_NOTEBOOK_MODE
    
    if toggle:
        print('Enabling inline hypertools plots')
        init_notebook_mode(connected=True)
        HYPERTOOLS_NOTEBOOK_MODE = True
    else:
        print('Disabling interactive hypertools plots')
        init_notebook_mode(connected=False)
        HYPERTOOLS_NOTEBOOK_MODE = False
    return None

In [5]:
hypertools_notebook_mode(True)

Enabling inline hypertools plots


In [6]:
reduce_models = ['DictionaryLearning', 'FactorAnalysis', 'FastICA', 'IncrementalPCA', 'KernelPCA', 'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
 'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder', 'SparsePCA', 'TruncatedSVD', 'UMAP', 'TSNE', 'MDS', 'SpectralEmbedding', 'LocallyLinearEmbedding', 'Isomap']
cluster_models = ['AffinityPropagation', 'AgglomerativeClustering', 'Birch', 'DBSCAN', 'FeatureAgglomeration', 'KMeans', 'MeanShift', 'MiniBatchKMeans', 'SpectralBiclustering', 'SpectralClustering', 'SpectralCoclustering', 'DBSCAN', 'AffinityPropagation', 'MeanShift']
mixture_models = ['GaussianMixture', 'BayesianGaussianMixture', 'LatentDirichletAllocation', 'NMF']
decomposition_models = ['LatentDirichletAllocation', 'NMF']
text_vectorizers = ['CountVectorizer', 'TfidfVectorizer']
interpolation_models = ['linear', 'time', 'index', 'pad', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial']
text_models = ['USE', 'LatentDirichletAllocation', 'NMF']
align_models = ['srm', 'hyper', 'procrustes']
corpora = ['wiki', 'nips', 'sotus']
use_corpora = [str(k) for k in defaults['corpora'].keys()]

### API ideas

1. [x] Write a single function to format data into a list of pandas dataframes:
    - [x] If text, should vectorize text according to default (or given) models.  Allow user to (optionally) pass in row/column labels for the text data.  Each string gets one row; a list of strings yields a len(list) by n-features dataframe
    - [x] If numpy arrays, set indices to row count and columns to column count
    - [x] If pandas dataframe, keep as is but turn into a list
    - [x] If only a single matrix is given, still return a list
    - [x] Dimensions may be mismatched
    - [x] Missing data is left unchanged
2. [x] Write a function to take a list of dataframes and use PPCA to fill in missing values.  Also use interpolation to fill in missing rows
3. [x] Write a reduce function that can work in batch mode (reduce a single dataframe or a list of dataframes)
4. [x] Write a cluster function that can work in batch mode (return cluster labels or mixture proportions for a single dataframe or a list of dataframes)
5. [x] Write a text-vectorizer function that wraps universal sentence encoder, LDA, NMF, etc.
6. Write an align function that aligns all dataframes in a list into a common space
7. [x] Write a normalize function that z-scores the data
8. Write a plot function that takes a list of dataframes and plots them
    - Need a mat2colors function that colorizes data points according to group labels, cluster assignments, mixture proportions, or user-specified matrices.  Take in a colormap as an argument
    - Need support for multicolored lines in addition to per-observation marker colors.  For 3d plots, could use streamtubes...but for 2d plots this won't work, so maybe a segment solution would be appropriate.
    - Potentially allow for different line/marker colors
    - Support for animations:
      - Spin
      - Sliding window
      - Sliding window with tail
      - Sliding window with head
    - Support for multiindex dataframes (hierarchical plots with different coloring/line thickness)
9. Write a hyper-function that takes in a "messy" dataset, applies formatting, PPCA, reduce, alignment, and clustering (as specified) and returns a formatted/processed dataset.
10. Add hyper-function call to every helper function, so that all analyses can be applied from any call to any other function

In [7]:
class TransformableDataFrame(pd.DataFrame):
    
    def reduce(self):
        self.reduce_params = {}
        pass
    
    def normalize(self):
        self.normalize_params = {}
        pass
    
    def embed(self): #text embedding
        self.embed_params = {}
        pass
    
    def align(self):
        self.align_params = {}
        pass
    
    def plot(self):
        pass
    
    def invert(self):
        #invert alignment
        #invert reduction
        #invert normalization
        #invert text embedding
        
        pass


In [8]:
def is_dataframe(x):
    return type(x).__module__ in ['pandas.core.frame', 'modin.pandas.dataframe']

def is_multiindex_dataframe(x):
    return is_dataframe(x) and (type(x.index).__module__ == 'pandas.core.indexes.multi')

def is_array(x):
    return (not ('str' in str(type(x)))) and (type(x).__module__ == 'numpy')

def is_empty(x):
    return (x is None) or (len(x) == 0)

def is_text(x):
    if type(x) == list:
        return np.all([is_text(t) for t in x])
    return (type(x) in six.string_types) or (type(x) == np.str_)

In [9]:
def to_str_list(x, encoding='utf-8'):
    def to_string(s):
        if type(s) == str:
            return s
        elif is_empty(s) or (s == None):
            return ''
        elif type(s) in [bytes, np.bytes_]:
            return s.decode(encoding)
        elif is_array(s) or is_dataframe(s) or (type(s) == list):
            if len(s) == 1:
                return to_string(s[0])
            else:
                return to_str_list(s, encoding=encoding)
        else:
            return str(s)
    
    if is_array(x) or (type(x) == list):        
        return [to_string(s) for s in x]
    elif is_text(x):
        return [x]
    else:
        raise Exception('Unsupported data type: {type(x)}')

In [10]:
def get_corpus(c, encoding='utf-8'):
    import hypertools as hyp
    if c in corpora:
        fname = os.path.join(eval(defaults['data']['datadir']),'corpora', f'{c}.npy')
        if not os.path.exists(fname):
            if not os.path.exists(os.path.abspath(os.path.join(fname, os.pardir))):
                os.makedirs(os.path.abspath(os.path.join(fname, os.pardir)))
            corpus_words = to_str_list(hyp.load(c).data[0]) #TODO: FIX THIS TO NOT CALL HYPERTOOLS!
            
            np.save(fname, corpus_words)
            return corpus_words
        else:
            corpus_words = np.load(fname, allow_pickle=True)
            return to_str_list(corpus_words)
    else:
        if is_text(c):
            if type(c) == list:
                return c
            else:
                return [c]
        elif os.path.exists(c):
            return to_str_list([x[0] for x in np.load(c, allow_pickle=True).tolist()])
        else:
            raise Exception(f'Unknown corpus: {c}')

In [11]:
def vectorize_text(text, vectorizer='CountVectorizer', vocabulary=None, return_vocab=False):
    if not (type(text) == list):
        text = [text]
    assert is_text(text), f'Must vectorize a string or list of strings (given: {type(text)})'
    
    if type(vectorizer) in six.string_types:
        assert vectorizer in text_vectorizers, f'Text vectorizer must be a function or a member of {text_vectorizers}'
        vectorizer = eval(vectorizer)
    assert callable(vectorizer), f'Text vectorizer must be a function or a member of {text_vectorizers}'
    
    text2vec = vectorizer(max_df=eval(defaults['text']['max_df']),
                          min_df=eval(defaults['text']['min_df']),
                          stop_words=defaults['text']['stop_words'],
                          strip_accents=defaults['text']['strip_accents'],
                          lowercase=eval(defaults['text']['lowercase']),
                          vocabulary=vocabulary)
    vectorized_text = text2vec.fit_transform(text)
    
    if not return_vocab:
        return vectorized_text
    else:
        vocab = text2vec.get_feature_names()
        return vectorized_text, vocab

In [12]:
def get_text_model(corpus, model, vectorizer, n_components=50):
    if type(model) in six.string_types:
        assert model in text_models, f'Text model must be a function or a member of {text_models}'
        model = eval(model)
    assert callable(model), f'Text model must be a function or a member of {text_models}'
    
    if type(vectorizer) in six.string_types:
        assert vectorizer in text_vectorizers, f'Text vectorizer must be a function or a member of {text_vectorizers}'
        vectorizer = eval(vectorizer)
    assert callable(vectorizer), f'Text vectorizer must be a function or a member of {text_vectorizers}'
    
    if corpus in corpora:
        saveable = True
    else:
        if not os.path.exists(corpus):
            assert is_text(corpus), f'Corpus must be a list of strings, or one of {corpora}'
        saveable = False
    
    if saveable:
        fname = os.path.join(eval(defaults['data']['datadir']),'text-models', model.__name__, f'{corpus}-{vectorizer.__name__}-{n_components}.npz')    
        if not os.path.exists(os.path.abspath(os.path.join(fname, os.pardir))):
            os.makedirs(os.path.abspath(os.path.join(fname, os.pardir)))
    
    if saveable and os.path.exists(fname):
        with np.load(fname, allow_pickle=True) as x:
            return {'vocab': x['vocab'].tolist(), 'model': x['model'].tolist()}
    else:
        corpus = get_corpus(corpus)
        vectorized_corpus, vocab = vectorize_text(corpus, vectorizer=vectorizer, return_vocab=True)
        
        if n_components == None:
            n_components = eval(defaults['text']['topics'])
        args = {'n_components': n_components,
                'max_iter': eval(defaults['text']['max_iter'])}
        
        if model.__name__ == 'NMF' and (args['n_components'] > len(corpus)):
            args['n_components'] = len(corpus)
        
        if model.__name__ == 'LatentDirichletAllocation':
            args['learning_method'] = defaults['text']['learning_method']
            args['learning_offset'] = eval(defaults['text']['learning_offset'])
        
        #return args, vectorized_corpus, vocab
        
        embeddings = model(**args).fit(vectorized_corpus)
        
        if saveable:
            np.savez(fname, vocab=vocab, model=embeddings)
        
        return {'vocab': vocab, 'model': embeddings}        

In [13]:
def text_vectorizer(text, model='USE', **kwargs):
    warnings.simplefilter('ignore')
    def USE(text, **kwargs):
        if 'USE_corpus' in kwargs.keys():
            corpus = kwargs['USE_corpus']
        else:
            corpus = defaults['corpora'][defaults['text']['USE_corpus']]
        
        model = hub.load(corpus)        
        return np.array(model(text))
    
    def sklearn_vectorizer(text, model, **kwargs):                
        if 'corpus' in kwargs.keys():
            corpus = kwargs['corpus']
        else:
            corpus = defaults['text']['corpus']            
        
        assert (corpus in corpora) or is_text(corpus) or os.path.exists(corpus), f'Cannot use corpus: {corpus}'
        
        if 'vectorizer' in kwargs.keys():
            vecterizer = kwargs['vectorizer']
            kwargs.pop('vectorizer', None)
        else:
            vectorizer = defaults['text']['vectorizer']
        
        model = get_text_model(corpus, model, vectorizer)
        return model['model'].transform(vectorize_text(text, vectorizer=vectorizer, vocabulary=model['vocab']))
    
    assert (model in text_models) or (callable(model)), f'Unsupported model: {model}'
    if not (type(text) == list):
        text = [text]
    
    if callable(model):
        return model(text, **kwargs)
    elif model == 'USE':
        return USE(text, **kwargs)
    else:
        return sklearn_vectorizer(text, model, **kwargs)
    

In [14]:
def panda_stack(data, force=True, names=None, keys=None, verify_integrity=False, sort=False, copy=True, ignore_index=False, levels=None, **kwargs):
    '''
    Take a list of DataFrames with the same number of columns and (optionally)
    a list of names (of the same length as the original list; default:
    range(len(x))).  Return a single MultiIndex DataFrame where the original
    DataFrames are stacked vertically, with the data names as their level 1
    indices and their original indices as their level 2 indices.
    
    INPUTS
    data: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe.  If
    force is False, must pass in a list of DataFrames.
    
    force: if True, use format_data to coerce everything into a list of pandas dataframes.
    
    text_vectorizer: function for turning text data into DataFrames, used if force is True
    
    Also takes all keyword arguments from pandas.concat except axis, join, join_axes
    
    All other keywork arguments (if any) are passed to text_vectorizer
    
    OUTPUTS
    a single MultiIndex DataFrame
    '''
    
    if force:
        data = format_data(data, **kwargs)
    
    assert type(data) == list, 'Must either pass a list of DataFrames or set force to True'
    assert np.all([is_dataframe(d) for d in data]), 'Must either pass a list of DataFrames or set force to True'
    assert len(np.unique([d.shape[1] for d in data])) == 1, 'All DataFrames must have the same number of columns'    
    template = data[0].columns.values
    for i, d1 in enumerate(data):
        template = d1.columns.values
        for d2 in data[(i+1):]:
            assert np.all([(c in template) for c in d2.columns.values]), 'All DataFrames must have the same columns'
    
    if keys is None:
        keys = np.arange(len(data), dtype=np.int)
    
    assert is_array(keys) or (type(keys) == list), f'keys must be None or a list or array of length len(data)'
    assert len(keys) == len(data), f'keys must be None or a list or array of length len(data)'
    
    return pd.concat(data, axis=0, join='outer', join_axes=None, names=names, keys=keys, 
                     verify_integrity=verify_integrity, sort=sort, copy=copy, ignore_index=ignore_index, levels=levels)

In [15]:
def panda_unstack(x):
    if not is_multiindex_dataframe(x):
        if is_dataframe(x):
            return x
        else:
            raise Exception(f'Unsupported datatype: {type(x)}')
    
    names = list(x.index.names)
    grouper = 'ID'
    if not (grouper in names):
        names[0] = grouper
    elif not (names[0] == grouper):
        for i in np.arange(len(names)): #trying n things other than 'ID'; at least one of them must be outside of the n-1 remaining names
            next_grouper = f'{grouper}{i}'
            if not (next_grouper in names):
                names[0] = next_grouper
                grouper = next_grouper
                break
    assert names[0] == grouper, 'Unstacking error'
    
    x.index.rename(names, inplace=True)    
    unstacked = [d[1].set_index(d[1].index.get_level_values(1)) for d in list(x.groupby(grouper))]
    if len(unstacked) == 1:
        return unstacked[0]
    else:
        return unstacked

In [16]:
def format_data(x, **kwargs):
    '''
    INPUTS
    x: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe
    text_kwargs: dictionary of keywork arguments to pass to text_vectorizer
    
    OUTPUTS
    a list of pandas dataframes
    '''
    
    if 'text_kwargs' in kwargs.keys():
        text_kwargs = kwargs.pop('text_kwargs', None)
    else:
        text_kwargs = {}

    def to_dataframe(y):
        if is_dataframe(y):
            return y
        elif is_array(y):
            assert np.ndim(y) == 2, 'all data matrices must be 2-dimensional'
            idx = np.arange(y.shape[0])
            cols = np.arange(y.shape[1])
            return pd.DataFrame(data=y, index=idx, columns=cols)        
        elif is_text(y):    
            return panda_stack(text_vectorizer(y, **text_kwargs)).reset_index(drop=True)
            #else:
            #    return to_dataframe(text_vectorizer(y, **text_kwargs))
        elif type(y) == list:
            return [to_dataframe(i) for i in y]
        else:
            raise NotImplementedError(f'Unsupported data type: {type(y)}')
    
    if type(x) == list:
        return [to_dataframe(i) for i in x]
    else:
        return [to_dataframe(x)]

In [17]:
def fill_missing(x, apply_ppca=True, interpolation='linear', **kwargs):
    if type(x) == list:
        return [fill_missing(i, apply_ppca=apply_ppca, interpolation=interpolation, **kwargs) for i in x]
    elif is_array(x):
        return fill_missing(pd.DataFrame(x), apply_ppca=apply_ppca, interpolation=interpolation, **kwargs).values
    elif not is_dataframe(x):
        raise Exception('Unsupported datatype: f{type(x)}')
    
    if 'method' in interpolate_kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {interpolate_kwargs['method']}")
    else:
        interpolate_kwargs['method'] = interpolation
    
    assert interpolate_kwargs['method'] in interpolation_models, f"Unsupported interpolation type: '{interpolate_kwargs['method']}'"
    
    if apply_ppca:
        covariance_model = PPCA()
        covariance_model.fit(x.values)
        x.values = covariance_model.transform()
    
    if not ('inplace' in kwargs.keys()):
        kwargs['inplace'] = False
    
    if not ('axis' in kwargs.keys()):
        kwargs['axis'] = 0
        
    if 'method' in kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {kwargs['method']}")
    else:
        kwargs['method'] = interpolation
    
    return x.interpolate(**kwargs)

In [18]:
def format_interp_stack_extract(data, keys=None, fillna=False, interpolation_kwargs={}, **kwargs):
    data = format_data(data, **kwargs)    
    
    if fillna:
        data = fill_missing(data, **interpolation_kwargs)
    
    stacked_data = panda_stack(data, keys=keys)
    vals = stacked_data.values
    return vals, stacked_data

In [19]:
def align(data, stack=False, keys=None, fillna=False, interpolation_kwargs={}, **kwargs):
    '''
    ARGUMENTS:
    data: data to reduce (numpy array or compatable, or a pandas
          dataframe or compatable).  Formatted as a 2d matrix whose
          rows are observations and whose columns are feature
          dimensions.  Can also input a list of Note: DataFrame indices are ignored; all DataFrames
          are aligned to the first r rows, where r is the number of rows
          in the shortest member of data.
    
    algorithm: one of: 'hyper', 'srm', 'procrustes'  Can also
          pass a function directly.
    
    fillna: if True, use PPCA and interpolation to fill in nan-valued entries (default: False)
    
    interpolation_kwargs: keyword arguments passed to the interpolation function, used
           when fillna is True.
    
    stack: if True, create a single (stacked) MultiIndex DataFrame out of
           the inputted data list and return a version of the same dataframe,
           but with re-named columns.
    
    keys: a name for each data matrix (default: None; name each set of observations
          range(len(data))).  Only relevant when stack is True.
    
    all additional keyword arguments are passed to the alignment function
    
    RETURNS:
    pandas dataframe (or list of dataframes) with number-of-observations rows and
    c columns, where c is the widest dataset in the list.
    '''
    
    #ignore reduce warnings
    warnings.simplefilter('ignore')
    
    if not ('algorithm' in kwargs.keys()):
        algorithm = defaults['align']['algorithm']
    else:
        algorithm = kwargs.pop('algorithm', 'None')
    
    #security check to prevent executing arbitrary code
    if is_text(algorithm):
        assert algorithm in align_models, f'Unknown algorithm: {algorithm}' 
        algorithm = eval(algorithm)
    else:
        assert algorithm.__module__ == 'align', f'Unknown algorithm: {algorithm}'
    
    _, stacked_data = format_interp_stack_extract(data, keys, fillna, interpolation_kwargs, **kwargs)
    unstacked = panda_unstack(stacked_data)
    
    aligned = algorithm([x.values for x in unstacked], **kwargs)
    
    if stack:
        return pd.DataFrame(data=np.vstack(aligned), index=stacked_data.index)
    else:
        return [pd.DataFrame(data=v, index=unstacked[i].index) for i, v in enumerate(aligned)]

#TODO: debug this (and functions in align.py)

In [20]:
def reduce(data, stack=False, keys=None, fillna=False, interpolation_kwargs={}, **kwargs):
    '''
    ARGUMENTS:
    data: data to reduce (numpy array or compatable, or a pandas
          dataframe or compatable).  Formatted as a 2d matrix whose
          rows are observations and whose columns are feature
          dimensions.
    
    algorithm: one of: 'DictionaryLearning', 'FactorAnalysis',
          'FastICA', 'IncrementalPCA', 'KernelPCA',
          'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
          'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder',
          'SparsePCA', 'TruncatedSVD', or 'UMAP'.  Calls the
          relevant scikit-learn or UMAP function.  Can also
          pass a function directly.
    
    ndims: the number of dimensions (columns) in the result (default: 3).
          if ndims > data.shape[1], the right-most columns will be
          zero-padded.
    
    fillna: if True, use PPCA and interpolation to fill in nan-valued entries (default: False)
    
    interpolation_kwargs: keyword arguments passed to the interpolation function, used
           when fillna is True.
    
    stack: if True, create a single (stacked) MultiIndex DataFrame out of
           the inputted data list and return a version of the same dataframe,
           but with ndims columns (named range(ndims))
    
    keys: a name for each data matrix (default: None; name each set of observations
          range(len(data))).  Only relevant when stack is True.
    
    all additional keyword arguments are passed to the reduce function
    
    RETURNS:
    pandas dataframe (or list of dataframes) with number-of-observations rows and
    ndims columns
    '''
    
    #ignore reduce warnings
    warnings.simplefilter('ignore')
    
    if not ('algorithm' in kwargs.keys()):
        algorithm = defaults['reduce']['algorithm']
    else:
        algorithm = kwargs.pop('algorithm', 'None')
    
    if not ('ndims' in kwargs.keys()):
        ndims = eval(defaults['reduce']['ndims'])
    else:
        ndims = kwargs.pop('ndims', 'None')
    
    if is_text(algorithm):
        assert algorithm in reduce_models, f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
        algorithm = eval(algorithm)
    else:
        if not (('sklearn.decomposition' in algorithm.__module__) or ('umap.umap_' in algorithm.__module__)):
            raise Exception(f'Unknown algorithm: {algorithm}')
    
    assert ndims >= 0, f'Number of dimensions must be non-negative (given: {ndims})'
    
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    
    vals, stacked_data = format_interp_stack_extract(data, keys, fillna, interpolation_kwargs, **kwargs)
    
    if vals.shape[1] > ndims:
        n_obs = np.sum([x.shape[0] for x in data])
        assert n_obs > ndims, f'Total number of observations must be greater than the number of dimensions; given n_obs: {n_obs}, ndims: {ndims}'
        
        model = algorithm(n_components=ndims, **kwargs)
        reduced_vals = model.fit_transform(vals)
    elif vals.shape[1] < ndims:
        reduced_vals = np.concatenate([vals, np.zeros([vals.shape[0], ndims-vals.shape[1]])], axis=1)
    else:
        reduced_vals = vals
    
    reduced_data = pd.DataFrame(data=reduced_vals, index=stacked_data.index, columns=np.arange(ndims))
    if stack:
        return reduced_data
    else:
        return panda_unstack(reduced_data)

In [21]:
def normalize(data, stack=False, keys=None, fillna=False, interpolation_kwargs={}, **kwargs):
    vals, stacked_data = format_interp_stack_extract(data, keys, fillna, interpolation_kwargs, **kwargs)
    
    vals = (vals - np.mean(vals, axis=0)) / np.std(vals, axis=0)
    stacked = pd.DataFrame(data=vals, index=stacked_data.index, columns=stacked_data.columns)
    
    if stack:
        return stacked
    else:
        return panda_unstack(stacked)

In [22]:
def cluster(data, algorithm='KMeans', k=10, stack=False, keys=None, fillna=False, interpolation_kwargs={}, return_model=False, **kwargs):
    if is_text(algorithm):
        assert (algorithm in cluster_models) or (algorithm in mixture_models), f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
        algorithm = eval(algorithm)
    else:
        if not (('sklearn.cluster' in algorithm.__module__) or ('sklearn.decomposition' in algorithm.__module__) or ('sklearn.mixture' in algorithm.__module__)):
            raise Exception(f'Unknown algorithm: {algorithm}')
    
    k_dict = {'cluster_models': 'n_clusters', 'mixture_models': 'n_components'}
    for model_type in k_dict.keys():
        if algorithm.__name__ in eval(model_type):
            if not (k_dict[model_type] in kwargs.keys()):
                kwargs[k_dict[model_type]] = k
            else:
                warnings.warn(f"Overloaded keyword argument; ignoring k value ({k}) in favor of keyword-specified {k_dict[model_type]}: {kwargs[k_dict[model_type]]}")
            break
    
    if not ('verbose' in kwargs.keys()):
        kwargs['verbose'] = eval(defaults['cluster']['verbose'])
    
    vals, stacked_data = format_interp_stack_extract(data, keys, fillna, interpolation_kwargs, **kwargs)
    
    if algorithm.__name__ in decomposition_models:
        if np.min(vals) < 0:
            vals -= np.min(vals) #avoid negative values by setting minimum value to 0
    
    model = algorithm(**kwargs)
    model.fit(vals)
    
    if hasattr(model, 'n_clusters'):
        n = 1
    elif hasattr(model, 'n_components'):
        n = model.n_components
    else:
        raise NotImplementedError(f'Unsupported model: {model}')
    
    if hasattr(model, 'predict_proba'):
        labels = model.predict_proba(vals)
    elif hasattr(model, 'predict'):
        labels = model.predict(vals)
    elif hasattr(model, 'transform'):
        labels = model.transform(vals)
    else:
        raise NotImplementedError(f'Unsupported model: {model}')
    
    labels = np.array(labels, ndmin=2)
    if n == 1:
        labels = labels.T
    
    stacked_labels = pd.DataFrame(data=labels, index=stacked_data.index, columns=np.arange(n))
    
    if stack:
        if return_model:
            return stacked_labels, model
        else:
            return stacked_labels
    else:
        if return_model:
            return panda_unstack(stacked_labels), model
        else:
            return panda_unstack(stacked_labels)

In [23]:
xs = [np.cumsum(np.random.randn(100, 10), axis=0) for i in np.arange(20)]

In [24]:
x_stacked = panda_stack(xs)

In [25]:
x_stacked.head(20)

0         1         2         3         4         5         6  \
0 0   -2.278452 -0.867202  0.630037  1.775311  0.002937  1.407164  0.021669   
  1   -3.535950 -0.996718  0.635439  0.146586 -1.723358  1.875210 -1.406902   
  2   -5.055359 -2.195433  1.450172  0.456257 -3.155258  1.514447 -1.634752   
  3   -5.030867 -2.707294  0.576853  1.814495 -0.984568  2.880238 -2.466645   
  4   -4.866484 -2.304972  0.201201  3.077494 -2.212322  3.606995 -2.479302   
  5   -4.937312 -2.011818 -0.138041  3.362955 -0.016840  3.805688 -0.990021   
  6   -3.665100 -2.436665  0.446776  3.942119 -1.322945  1.957467  0.536351   
  7   -4.462918  0.005977  0.451377  4.024556 -1.175106  1.901043  0.279984   
  8   -3.750189 -0.510387  1.134948  4.879226 -1.921668  1.748672  2.709438   
  9   -4.749960 -0.741560 -0.713466  4.954622 -1.959211  1.614662  4.769680   
  10  -4.221030 -2.448541 -2.658392  4.479962 -1.872163  0.452582  5.574069   
  11  -5.728185 -3.330062 -2.088481  3.826970 -1.723386 -0.376029  7.428736   
  12  -7.857911 -3.788535 -1.150544  5.164013 -1.610691  0.178145  7.974781   
  13  -8.258913 -1.898007  0.505740  7.268677 -0.933680  1.451490  7.883715   
  14  -8.225565 -0.527313  0.133969  7.373584 -0.208085  0.588499  6.127327   
  15  -8.002441  0.006034  0.554565  6.202618 -0.564663  1.184265  6.286525   
  16 -10.706706 -0.517968  0.331845  6.029823  1.494669  1.590593  6.638155   
  17 -10.200758 -1.556848  1.147354  5.140273  1.318711  1.137989  9.279124   
  18  -9.515100 -2.694554  1.789255  4.524429  1.588533  0.809807  9.993030   
  19  -8.573782 -2.878527  1.823800  3.870978  1.345828  1.156944  8.122190   

             7         8         9  
0 0  -1.987936 -0.317783  0.052343  
  1  -0.092239  1.591359  1.308667  
  2  -0.953721  1.737676  2.343546  
  3  -0.811155  1.446512  0.665637  
  4   1.200674  2.266069 -1.423163  
  5   0.565329  0.740183 -1.149960  
  6   1.673519 -0.131892 -2.257478  
  7   1.889874 -1.552112 -1.231944  
  8   2.375820 -1.407667 -1.243556  
  9   0.169239  0.649663 -2.914711  
  10  1.241692  0.398485 -3.785935  
  11  1.808420  0.197034 -3.903259  
  12  0.323947  1.131370 -3.548513  
  13  0.353220 -0.233857 -3.319501  
  14  1.805142 -1.963452 -2.366271  
  15  1.033579 -1.913433 -2.328304  
  16  1.440715 -0.899995 -1.700275  
  17  1.886733 -1.568675 -1.455020  
  18  1.912454 -1.981425 -1.679508  
  19  1.011223 -2.102266 -1.782479

In [26]:
cluster(xs, stack=True, algorithm='GaussianMixture', k=4)

0             1             2             3
0  0   0.080783  5.835067e-09  9.192144e-01  2.263542e-06
   1   0.187331  5.926792e-08  8.126685e-01  2.385769e-08
   2   0.300409  3.076533e-11  6.995888e-01  2.558923e-06
   3   0.467667  1.509349e-10  5.323330e-01  2.855848e-08
   4   0.553971  1.291703e-08  4.460294e-01  3.616595e-09
...         ...           ...           ...           ...
19 95  1.000000  1.163611e-53  2.226904e-17  2.047238e-63
   96  1.000000  2.241410e-52  5.948429e-17  2.568326e-61
   97  1.000000  7.044145e-51  1.645817e-16  4.277256e-52
   98  1.000000  1.126014e-54  5.798568e-18  2.890995e-56
   99  1.000000  6.363067e-59  3.199810e-19  4.518120e-62

[2000 rows x 4 columns]

In [27]:
def plotter(data, style='line', name=None, reduce_kwargs={}, text_kwargs={}, suppress_plot=False, **kwargs):
    global HYPERTOOLS_NOTEBOOK_MODE
        
    def combo_merge(a, b):
        combos = []
        for i in a:
            for j in b:
                if (len(i) <= 2):
                    if (len(j) <= 2):
                        combos.append(i+j)
                elif len(j) > 1:
                    combos.append(i + '+' + j)
        return combos
    
    marker_styles = ['.', 'o', 'scatter', 'marker', 'markers', 'bigmarker', 'bigmarkers']
    line_styles = ['-', '--', ':', '-:', 'line', 'lines']
    combo_styles = combo_merge(marker_styles, line_styles) + combo_merge(line_styles, marker_styles)
    big_markers = ['o', 'big']
    dash_styles = {'--': 'dash', '-:': 'dashdot', ':': 'dot'}
    
    def substr_list(style, x):
        '''
        style: a style string
        x: a list of substrings
        
        return: true if any of the strings in x is a substring of s, and false othewise    
        '''
        inds = np.array([s in style for s in x])
        if np.any(inds):
            return x[np.where(inds)[0][0]]
        else:
            return ''
    
    is_line = lambda s: substr_list(s, line_styles + combo_styles)
    is_marker = lambda s: substr_list(s, marker_styles + combo_styles)
    is_combo = lambda s: substr_list(s, combo_styles)
    
    is_dashed = lambda s: substr_list(s, list(dash_styles.keys()))
    is_bigmarker = lambda s: substr_list(s, big_markers)
    
    data = format_data(data, **text_kwargs) #process text data (if needed)
    
    if len(data) > 1:        
        reduced_data = reduce(data, stack=True, **reduce_kwargs) #also calls format_data, but without text_kwargs
        split_data = panda_unstack(reduced_data)
        names = [str(n) for n in np.array(reduced_data.index.levels[0])]
    else:
        reduced_data = reduce(data, **reduce_kwargs)
        split_data = [reduced_data]
        names = [str(0)]
    
    if reduced_data.shape[1] == 2:
        suffix = ''
    elif reduced_data.shape[1] == 3:
        suffix = '3D'
    else:
        raise NotImplementedError(f'Unsupported number of dimensions; reduced data must be 2d or 3d, given: {reduced_data.shape[1]}')
    
    #TODO: add support for multiple styles (up to one per line)
    #TODO: load in custom plot theme/style from defaults, also allow user to customize
    #TODO: draw bounding box around space
    #TODO: support animation:
    # - camera rotation
    # - window length around current timepoint
    # - opacity of prior timepoints
    # - opacity of current window
    # - opacity of future timepoints    
    #TODO: legend customization
    #TODO: label each point with its original values and/or a user-specified label
    #TODO: support per-datapoint colors, including for lines
    #TODO: support for mixture-based colors-- if 1d, map onto colormap.  If 1 < n-colors <= 3, pick
    #      a color for each dimension and then compute each coordinate as a weighted blend.  If > 3,
    #      map onto 3D colors and then treat as 3D.  Can also define custom functions-- should take
    #      in any observation and return a new vector describing its color (in >= 1D).  also support
    #      a list of functions (one per data matrix)
    #TODO: support plotting of error bars via streamtubes and/or ribbon plots
    #TODO: support plotting "vector fields" by taking a list of two matrices of coordinats,
    #      one specifying the start points and the other specifying the end points of each
    #      vector
    
    #Write a general "hypertools function" that does all analyses by calling normalize --> align --> reduce
    #in sequence.
    
    #TODO: support for nested (>2 level dataframes)
    #TODO: support interpolation for line_styles and combo_styles
    #TODO: define default options outside of this function in a dictionary
    #TODO: normalization options: z-score down rows, across columns, both, or arbitrary functions (take
    #      in a data matrix and return a new matrix of the same type/size).  can also provide lists of
    #      normalization functions to be applied separately to each data matrix.
    #TODO: add support for multiple reduce functions (and/or reduce_kwargs)-- one per data matrix
    #TODO: add hyperalignment and SRM.  also allow different subsets of the data to be aligned, e.g.
    #      according to the unique IDs specified in a list of group IDs, some column of the data matrix,
    #      cluster IDs, or a custom function (or a dictionary of functions specifying how different
    #      group IDs should be aligned); functions should take in a list of data matrices and returned an aligned
    #      list in the same format.
    
    style = style.lower()
    
    assert style in marker_styles + line_styles + combo_styles, f'Unsupported plot style: {style}'
    
    if is_combo(style):
        mode='lines+markers'
    elif is_line(style):
        mode='lines'
    else:
        mode='markers'
    
    if is_line(style):
        if not('line' in kwargs.keys()):
                kwargs['line'] = {}
        #if not('opacity' in kwargs['line'].keys()):
        #    kwargs['line']['opacity'] = float(defaults['plot']['opacity'])
        
        dash = is_dashed(style)
        if dash:
            #print(f'dashed line: {dash}')
            if not('dash' in kwargs['line'].keys()):
                kwargs['line']['dash'] = dash_styles[dash]
    if is_marker(style):
        if is_bigmarker(style):
            size = int(defaults['plot']['bigmarkersize'])
        else:
            size = int(defaults['plot']['markersize'])
        
        if not('marker' in kwargs.keys()):
            kwargs['marker'] = {}
        if not('size' in kwargs['marker'].keys()):
            if is_bigmarker(style):
                kwargs['marker']['size'] = int(defaults['plot']['bigmarkersize'])
            else:
                kwargs['marker']['size'] = int(defaults['plot']['markersize'])        
        if not('opacity' in kwargs['marker'].keys()):
            kwargs['marker']['opacity'] = float(defaults['plot']['opacity'])    
    
    if reduced_data.shape[1] == 2:
        shapes = [go.Scatter(x=d.iloc[:, 0], y=d.iloc[:, 1], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    elif reduced_data.shape[1] == 3:
        shapes = [go.Scatter3d(x=d.iloc[:, 0], y=d.iloc[:, 1], z=d.iloc[:, 2], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    
    if suppress_plot:
        return shapes
    else:
        if HYPERTOOLS_NOTEBOOK_MODE:
            return iplot(shapes)
        else:
            return plot(shapes)

In [28]:
x_r = reduce(xs, algorithm='UMAP', stack=True)
x_r

0         1         2
0  0    8.069575  4.241327  7.667076
   1    8.009190  4.085773  7.462676
   2    8.006886  4.074054  7.483866
   3    7.888070  4.012213  7.545573
   4    7.874285  4.013829  7.534245
...          ...       ...       ...
19 95  10.579613 -7.841271  3.179210
   96  10.579194 -7.843049  3.179705
   97  10.586412 -7.850205  3.172503
   98  10.582940 -7.851122  3.176063
   99  10.597048 -7.864813  3.162059

[2000 rows x 3 columns]

In [29]:
def animate3d(shapes, title='', transition_duration=500, width=600, height=600, bounds=[-10, 10]):

    n_frames = len(shapes)
    frames = [go.Frame(data=shapes[i], name=str(i)) for i in range(n_frames)]
    #ids = [[str(i) for i in range(len(s))] for s in shapes]

    fig = go.Figure(frames=frames)

    # Add data to be displayed before animation starts
    fig.add_trace(frames[0].data[0])

    def frame_args(duration):
        return {"frame": {"duration": duration},
                #"mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": duration, "easing": "cubic-in-out"}}

    sliders = [{"pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [{"args": [[f.name], frame_args(0)],
                           "label": str(k),
                           "method": "animate"} for k, f in enumerate(fig.frames)]}]

    # Layout
    fig.update_layout(title=title, width=width, height=height,
             scene=dict(zaxis=dict(range=[bounds[0], bounds[1]], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1)),
             updatemenus = [{"buttons": [{"args": [None, frame_args(transition_duration)],
                             "label": "&#9654;", # play symbol
                             "method": "animate"},
                            {"args": [[None], frame_args(0)],
                             "label": "&#9724;", # pause symbol
                             "method": "animate"}],
                            "direction": "left", "pad": {"r": 10, "t": 70},
                            "type": "buttons", "x": 0.1, "y": 0}],
             sliders=sliders)

    fig.show()
    

In [30]:
#look at this to make smoother animations: https://plotly.com/javascript/animations/#animating-many-frames-quickly

In [31]:
animate3d([plotter(xs[i], 'o', suppress_plot=True) for i in range(len(xs))]);

In [66]:
x = panda_unstack(x_r)

In [86]:
x[0].iloc[2:12, :]

,0,1,2
2,8.006886,4.074054,7.483866
3,7.888070,4.012213,7.545573
4,7.874285,4.013829,7.534245
5,7.813659,4.031681,7.577821
6,7.754359,4.023847,7.618953
7,7.730466,4.044007,7.686415
8,7.687333,4.029000,7.671685
9,1.018523,0.680777,6.410911
10,0.974765,0.646810,6.386685
11,0.963522,0.645313,6.386004


In [90]:
xs = [x[0]]
framesize = 10
shapes = [plotter([x.iloc[i:(i + framesize)] for x in xs], 'o', suppress_plot=True) for i in range(0, x[0].shape[0] - framesize)]

In [91]:
animate3d(shapes)
#STOPPED HERE...

In [ ]:
np.lib.stride_tricks.as_strided(x[0].values, )

In [72]:
x[0].rolling(window=10, min_periods=1).apply(lambda f: pd.DataFrame(plotter(f, 'o', suppress_plot=True)))

AssertionError: all data matrices must be 2-dimensional

In [32]:
p = plotter(xs[0])

In [33]:
plotter(xs, '.')

In [34]:
plotter(normalize(xs), 'o')

In [35]:
from align import hyper, srm, procrustes, trim_and_pad, pad_and_align
xs = [np.cumsum(np.random.randn(100, 10), axis=0) for i in np.arange(len(xs))]

In [36]:
xs_aligned = align(xs, algorithm='srm')

In [37]:
plotter(xs_aligned, style='-')

In [38]:
plotter(xs, style='-')

In [39]:
plotter(xs, style='-o')

In [40]:
plotter(xs, style='.', reduce_kwargs={'ndims': 2})

In [41]:
plotter(xs, style='-', reduce_kwargs={'ndims': 2})

In [42]:
sotus_corpus = get_corpus('sotus')

In [43]:
vec_sotus = format_data(sotus_corpus)

In [44]:
vectorized = format_data([sotus_corpus, ['this is a test', 'this is another test', 'raining cats and dogs', 'how much wood could a woodchuck chuck if a woodchuck could chuck wood?']]) #need to change format data to change a list of strings into a single matrix; a list of lists is a lists of matrices

In [45]:
plotter(vectorized, 'o:', reduce_kwargs={'algorithm': 'UMAP'})

In [46]:
text = [sotus_corpus, ['this is a test', 'this is another test', 'raining cats and dogs', 'how much wood could a woodchuck chuck if a woodchuck could chuck wood?']]

In [47]:
plotter(text, 'o:', reduce_kwargs={'algorithm': 'UMAP'}, text_kwargs={'model': 'LatentDirichletAllocation'})




In [48]:
plotter(['cats are the greatest animal', 'apples bananas pears plums', 'this is a test', 'hello how are you?'], 'o:', reduce_kwargs={'ndims': 2, 'algorithm': 'UMAP'}, text_kwargs={'model': 'NMF', 'corpus': 'nips'})



In [49]:
plotter(['cats are the greatest animal', 'apples bananas pears plums', 'this is a test', 'hello world', 'happy birthday to you'], 'o:', reduce_kwargs={'ndims': 3, 'algorithm': 'UMAP'}, text_kwargs={'model': 'NMF', 'corpus': 'nips'})



In [50]:
text2 = ['cats are the greatest animal', 'apples bananas pears plums', 'this is a test', 'happy birthday']

In [51]:
words = ['duck', 'goose', 'hen', 'origami', 'kitten', 'horse', 'paper', 'book', 'paint', 'toy', 'car', 'truck']

In [52]:
plotter(words, 'o')

In [53]:
vec_words = format_data(words)

In [54]:
plotter(vec_words, 'o', reduce_kwargs={'algorithm': 'SpectralEmbedding', 'ndims': 3})

In [55]:
vec2 = format_data(text2, text_kwargs={'model': 'LatentDirichletAllocation', 'corpus': 'nips'})

In [56]:
reduce(format_data(vec2), ndims=3, stack=True)

,,0,1,2
0,0,0.596817,0.146442,-0.039561
1,0,-0.131025,-0.374838,-0.310693
2,0,-0.099376,-0.197812,0.416569
3,0,-0.366417,0.426208,-0.066315


In [57]:
#note: doesn't work without specifying a marker-based line style.  should it?
plotter(vec2, 'o', reduce_kwargs={'ndims': 3})

In [58]:
#FIXME: this works, but should be the same as the previous cell
plotter(reduce(vec2, ndims=3), 'o')